## Real-time hand motion tracking towards surgical performance analysis

### Computer Vision
#### University Master's in Health Information Engineering - Universidad Carlos III

--------------------------------------------------

Isabel Gutiérrez Menzinger - 100464823@alumnos.uc3m.es

Violeta Sánchez García - 100384181@alumnos.uc3m.es

In [ ]:
import cv2
import mediapipe as mp
import time
import numpy as np

cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=1,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

pTime = 0
cTime = 0
coords = []
steps = []

while (cap.isOpened()):
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
        
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            
            myHand = results.multi_hand_landmarks[0].landmark
            myHand_x, myHand_y, myHand_z = np.round(myHand[-1].x, 3), np.round(myHand[-1].y, 3), np.round(myHand[-1].z, 3)
            
            disp_text1 = 'x: {}, y: {}, z: {}'.format(myHand_x, myHand_y, myHand_z)
            cv2.rectangle(img, (25, 25), (20 + 375, 50 + 35), (0,0,0), -1)
            cv2.putText(img, disp_text1, (30,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 1)
            
            coords.append([myHand_x, myHand_y, myHand_z])
            
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x *w), int(lm.y*h)
                cv2.circle(img, (cx,cy), 3, (255,0,0), cv2.FILLED)

            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
            
        for i in range(len(coords)-1):
            steps.append(np.linalg.norm([coords[i+1][0] - coords[i][0], 
                                         coords[i+1][1] - coords[i][1], 
                                         coords[i+1][2] - coords[i][2]]))
        
        disp_text2 = 'd: {}'.format(np.round(sum(steps), 3))
        cv2.putText(img, disp_text2, (30,75), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 1)
                                    
    else:
        disp_text = 'No elements found'
        cv2.rectangle(img, (25, 25), (20 + 240, 50 + 10), (0,0,0), -1)
        cv2.putText(img, disp_text, (30,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 1)

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.imshow("Hand Tracking", img)
    cv2.waitKey(1)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()